In [8]:
from IPython.display import display
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error
from scipy.stats import zscore
import gzip
import os
import csv
from alive_progress import alive_bar
from collections import defaultdict

types = defaultdict(int, token = "string")
numbers = list(range(1490,2020)) 
names = ['token']
names = names + numbers

directory = 'C:/Users/bincl/BA-Thesis/Dataset/2gram'
chunksize_per_df = 1000

In [2]:
def split_values(value):
    parts = value.split(',')
    if len(parts) < 2:
        return
    year = int(parts[0])
    val = int(parts[1])
    return pd.Series({'Year': year, str(year): val})

In [10]:

def transform_func(x):
    values = str(x).split(",")
    if len(values) < 2:
        return(0)
    return values[1]

In [11]:

df_chunk_list = pd.read_csv('C:/Users/bincl/BA-Thesis/Dataset/3gram/3_20000_nopos_sample.zip', compression= 'zip', header=None, converters = {col: transform_func for col in numbers}, index_col= [0],chunksize= chunksize_per_df, names = names,sep='\t', quoting=csv.QUOTE_NONE)
 

In [12]:
for chunk in df_chunk_list: 
    df_3gram_chunk = chunk 
    break 
display(df_3gram_chunk)

,1470,1471,1472,1473,1474,1475,1476,1477,1478,1479,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
token,,,,,,,,,,,,,,,,,,,,,
""" Bis seine",1,1,1,2,1,2,1,2,1,1,...,0,0,0,0,0,0,0,0,0,0
""" -- Vorgehen",2,2,1,1,1,1,1,1,1,2,...,0,0,0,0,0,0,0,0,0,0
""" : Oesterreich",1,1,1,1,6,1,1,1,2,3,...,0,0,0,0,0,0,0,0,0,0
""" Geht 's",1,1,2,1,2,4,9,1,1,1,...,0,0,0,0,0,0,0,0,0,0
""" Geschlecht '",1,3,1,3,1,2,1,2,2,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
""" 50. Und",1,1,1,1,1,1,4,1,3,1,...,0,0,0,0,0,0,0,0,0,0
""" 184 oder",1,1,2,2,1,2,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
""" Ansprüche des",1,1,2,1,1,1,1,5,1,1,...,0,0,0,0,0,0,0,0,0,0


In [ ]:

def find_n_gram(direc, find):
    for filename in os.listdir(direc):
        f = os.path.join(direc, filename)
        # checking if it is a file
        if os.path.isfile(f):
            print(f)
            with gzip.open(f,'rt', encoding='utf-8') as input:
                str_arr_csv = input.readlines()
                for line in str_arr_csv:
                    token = line.split("\t")
                    if find == token[0]:
                        return line
                        
    return "line not found" 

def start_the_search(directory,search):
    result = find_n_gram(directory,search)
    
    if result == "line not found":
        print(result)
        exit()
    result = result.split("\t")
    lower_ngram = pd.Series(result[1:],dtype= "string", name = result[0])
    return lower_ngram

In [ ]:
def getLowerLevelString(string):
   first, *middle, last = string.split()
   first = first + " " + ''.join(middle)
   last = ''.join(middle) + " " + last
   return first, last

def MLR(token,ngram):
    print(token)
    first,last = getLowerLevelString(token)
    print(first)
    print(last)
    first = start_the_search(directory,first)
    last = start_the_search(directory,last)
    
    X = pd.concat([first,last],axis=1)
    y = ngram.loc[token]
    reg = LinearRegression()
    reg.fit(X, y) 
    
    return reg.coef_,reg.intercept_,X

def buildApproximation(c1,c2,basevalue,df):
    df = df.astype('float')
    df['scaledFirst'] = df.iloc[:,0].apply(lambda x: x * c1) 
    df['scaledLast'] = df.iloc[:,1].apply(lambda x: x * c2) 
    df['approximation'] = df['scaledFirst'] + df['scaledLast'] + basevalue
    return df

In [ ]:
def compressWithError2gram(firstN,error,df_ngram):
    df_N_gram = df_ngram.head(firstN)
    
    result = []
    all = []
    sum = []
    with alive_bar(df_N_gram.index.values.size) as bar:
        for token in df_N_gram.index.values:
            coef,intercept,dfAprox = MLR(token,df_N_gram)
            c1,c2 = coef
            df = buildApproximation(c1,c2,intercept,dfAprox)
            dfOriginal = pd.DataFrame()
            dfOriginal['values'] = df_ngram.loc[token]
            dfOriginal['values'] = dfOriginal['values'].apply(pd.to_numeric, errors='coerce')
            #dfOriginal= pd.to_numeric(dfOriginal)
            dfOriginal['zscore'] = zscore(dfOriginal)
            df['zscore'] = zscore(df['approximation'])
            
            sum.append(pd.to_numeric(dfOriginal['values']).sum()) 
            
            rmse = mean_squared_error(dfOriginal['zscore'], df['zscore'], squared = False)
            #rmse = mean_squared_error(dfOriginal, df['approximation'])
            if rmse <= error:
                result.append([token,rmse,dfOriginal['values'],dfOriginal['zscore'],df['approximation'],df['zscore']]) 
            all.append(rmse)    
            bar()
    return result, all, sum


In [ ]:
result, all, sum = compressWithError2gram(chunksize_per_df,0.05, df_3gram_chunk)
print(len(result))


In [ ]:
plt.boxplot(all)
plt.show()
rmse_with_error = []

for i in result:
    rmse_with_error.append(i[1])
plt.boxplot(rmse_with_error)
plt.show()

import seaborn as sns

sns.violinplot(x= rmse_with_error, inner="point")

In [ ]:
sns.violinplot(x= all, inner_kws=dict(box_width=15, whis_width=2, color=".8"))

In [ ]:

plt.scatter(sum, all)
plt.xscale('log')
plt.show()